### Pre-Trained Transformers - BERT, RoBERTa, FinBERT


Out-of-Sample Performance (Macro Avg F1-Score)
* Twitter-RoBERTa-Base - 0.5300
* BERTweet-Base - 0.4800

In [2]:
! pip install transformers
! pip install emoji

     |████████████████████████████████| 4.0 MB 8.1 MB/s 
     |████████████████████████████████| 895 kB 65.6 MB/s 
     |████████████████████████████████| 77 kB 8.8 MB/s 
     |████████████████████████████████| 596 kB 47.7 MB/s 
     |████████████████████████████████| 6.5 MB 63.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 175 kB 8.9 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=22ada8138824bd189e89484ef7285490dfc94068eaa57198cba0aa1adfb81057
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji


In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('tweets_comments_combined_df.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4300 entries, 0 to 4299
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   comment    4300 non-null   object
 1   sentiment  4300 non-null   object
dtypes: object(2)
memory usage: 67.3+ KB


In [5]:
df['sentiment'].value_counts()

positive    1650
neutral     1337
negative    1313
Name: sentiment, dtype: int64

In [6]:
from sklearn.model_selection import train_test_split
X = df['comment']
y = df['sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42) 

In [7]:
# from transformers import pipeline

# # https://huggingface.co/ProsusAI/finbert
# model_path = "ProsusAI/finbert"
# sentiment_task = pipeline("sentiment-analysis", model=model_path, tokenizer=model_path)

# model_labels = []
# for comment in X_test:
#   sentiment = sentiment_task(comment)
#   model_labels.append(sentiment[0]['label'])

# from sklearn.metrics import classification_report
# print("Tuned Vader Sentiment Scores")
# print(classification_report(y_test, model_labels))

In [8]:
# #1-month Inference
# inference = pd.read_csv('inference_1month_comments.csv')

# import time
# t1 = time.perf_counter()

# model_labels = []
# for comment in inference['comment']:
#   sentiment = sentiment_task(comment)
#   model_labels.append(sentiment[0]['label'])

# t2 = time.perf_counter()
# print('time taken to run:', t2-t1)

In [9]:
from transformers import pipeline

# https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment-latest
model_path = "cardiffnlp/twitter-roberta-base-sentiment-latest"
sentiment_task = pipeline("sentiment-analysis", model=model_path, tokenizer=model_path)

model_labels = []
for comment in X_test:
  sentiment = sentiment_task(comment)
  model_labels.append(sentiment[0]['label'])

y_preds_labels = []
for label in model_labels:
  if label == 'Positive':
    y_preds_labels.append('positive')
  elif label == 'Neutral':
    y_preds_labels.append('neutral')
  elif label == 'Negative':
    y_preds_labels.append('negative')

from sklearn.metrics import classification_report
print(classification_report(y_test, y_preds_labels))

Downloading:   0%|          | 0.00/929 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

              precision    recall  f1-score   support

    negative       0.55      0.59      0.57       337
     neutral       0.42      0.66      0.51       322
    positive       0.77      0.39      0.52       416

    accuracy                           0.53      1075
   macro avg       0.58      0.55      0.53      1075
weighted avg       0.60      0.53      0.53      1075



In [10]:
# #1-month Inference
# inference = pd.read_csv('inference_1month_comments.csv')
# inference = inference[inference['comment'].map(len) <= 514]

# import time
# t1 = time.perf_counter()

# model_labels = []
# for comment in inference['comment']:
#   sentiment = sentiment_task(comment)
#   model_labels.append(sentiment[0]['label'])

# t2 = time.perf_counter()
# print('time taken to run:',t2-t1)

In [12]:
df_128 = df[df['comment'].map(len) <= 128] 
X_128 = df_128['comment']
y_128 = df_128['sentiment']

X_train_128, X_test_128, y_train_128, y_test_128 = train_test_split(X_128, y_128, test_size=0.25, random_state=42) 

# https://huggingface.co/finiteautomata/bertweet-base-sentiment-analysis

model_path = "finiteautomata/bertweet-base-sentiment-analysis"
sentiment_task = pipeline("sentiment-analysis", model=model_path, tokenizer=model_path)

model_labels = []
for comment in X_test_128:
  sentiment = sentiment_task(comment)
  model_labels.append(sentiment[0]['label'])

y_preds_labels = []
for label in model_labels:
  if label == 'POS':
    y_preds_labels.append('positive')
  elif label == 'NEU':
    y_preds_labels.append('neutral')
  elif label == 'NEG':
    y_preds_labels.append('negative')

from sklearn.metrics import classification_report
print(classification_report(y_test_128, y_preds_labels))

Downloading:   0%|          | 0.00/890 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/515M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/295 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/824k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/17.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

              precision    recall  f1-score   support

    negative       0.57      0.72      0.63       239
     neutral       0.35      0.58      0.44       209
    positive       0.77      0.24      0.37       295

    accuracy                           0.49       743
   macro avg       0.56      0.51      0.48       743
weighted avg       0.59      0.49      0.47       743



In [13]:
model_labels[:5]

['POS', 'NEU', 'NEG', 'NEU', 'NEU']

In [ ]:
# #1-month Inference
# inference = pd.read_csv('inference_1month_comments.csv')
# inference = inference[inference['comment'].map(len) <= 128]

# import time
# t1 = time.perf_counter()

# model_labels = []
# for comment in inference['comment']:
#   sentiment = sentiment_task(comment)
#   model_labels.append(sentiment[0]['label'])

# t2 = time.perf_counter()
# print('time taken to run:',t2-t1)